<a href="https://colab.research.google.com/github/vladcioaba/vault/blob/main/RoBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Set correct tensorflow
%tensorflow_version 2.x

import gast as gs
import folium as fl
import imgaug as ig
import numpy as np
import scipy as sp
import sklearn as sk
import pandas as pd
import platform as plat
import tensorflow as tf
import tkinter as tk
import torch as th

#how to check tools build versions
print('Python Version: ', plat.python_version())
#print('Gast Version: ',  gs.__version__)
print('Folium Version: ',  fl.__version__)
print('Imgaug Version: ',  ig.__version__)
print('NumPy Version: ',  np.__version__)
print('SciPy Version: ',  sp.__version__)
print('Scikit-Learn Version: ',  sk.__version__)
print('Pandas Version: ', pd.__version__)
print('Tensorflow Version: ', tf.__version__)
print('Tourch Version: ', th.__version__)
#print('tkinter test', tk._test())

Python Version:  3.6.9
Folium Version:  0.8.3
Imgaug Version:  0.2.9
NumPy Version:  1.19.5
SciPy Version:  1.4.1
Scikit-Learn Version:  0.22.2.post1
Pandas Version:  1.1.5
Tensorflow Version:  2.4.0
Tourch Version:  1.7.0+cu101


In [17]:
!git clone https://github.com/nishkalavallabhi/OneStopEnglishCorpus.git

Cloning into 'OneStopEnglishCorpus'...
remote: Enumerating objects: 3074, done.
remote: Total 3074 (delta 0), reused 0 (delta 0), pack-reused 3074
Receiving objects: 100% (3074/3074), 23.50 MiB | 21.56 MiB/s, done.
Resolving deltas: 100% (1143/1143), done.
Checking out files: 100% (2656/2656), done.


In [2]:
# Download roberta.large model
import os
if not os.path.exists("roberta.large"):
  !wget https://dl.fbaipublicfiles.com/fairseq/models/roberta.large.tar.gz && tar -xzvf roberta.large.tar.gz && rm roberta.large.tar.gz

In [3]:
!git clone https://github.com/pytorch/fairseq && cd fairseq && pip install --editable ./

fatal: destination path 'fairseq' already exists and is not an empty directory.


In [4]:
!pip install sacremoses
!pip install spacy
!pip install urllib3

In [5]:
import torch

roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
#roberta.cuda()
roberta.eval()  # disable dropout for evaluation

with torch.no_grad():
    # Encode a pair of sentences and make a prediction
    tokens = roberta.encode('Roberta is a heavily optimized version of BERT.', 'Roberta is not very optimized.')
    prediction = roberta.predict('mnli', tokens).argmax().item()
    assert prediction == 0  # contradiction

    # Encode another pair of sentences
    tokens = roberta.encode('Roberta is a heavily optimized version of BERT.', 'Roberta is based on BERT.')
    prediction = roberta.predict('mnli', tokens).argmax().item()
    assert prediction == 2  # entailment

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


In [35]:
# Batched prediction:
from fairseq.data.data_utils import collate_tokens
batch_of_pairs = [
  ['Outside is not raining', 'Outside is raining'],
  ['I like flowers', 'I not like flowers'],
  ['Violets are red', 'Roses are blue'],
  ['Mars is very far from earth.', 'Mars is very close.'],
]

batch = collate_tokens(
  [roberta.encode(pair[0], pair[1]) for pair in batch_of_pairs], pad_idx=1
)

logprobs = roberta.predict('mnli', batch)
print(logprobs.argmax(dim=1))

tensor([0, 0, 0, 0])


In [7]:
roberta.register_classification_head('new_task', num_classes=3)
logprobs = roberta.predict('new_task', tokens)  # tensor([[-1.1050, -1.0672, -1.1245]], grad_fn=<LogSoftmaxBackward>)

In [8]:
!!python -m spacy download en_core_web_lg

['Requirement already satisfied: en_core_web_lg==2.2.5 from https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz#egg=en_core_web_lg==2.2.5 in /usr/local/lib/python3.6/dist-packages (2.2.5)',
 'Requirement already satisfied: spacy>=2.2.2 in /usr/local/lib/python3.6/dist-packages (from en_core_web_lg==2.2.5) (2.2.4)',
 'Requirement already satisfied: thinc==7.4.0 in /usr/local/lib/python3.6/dist-packages (from spacy>=2.2.2->en_core_web_lg==2.2.5) (7.4.0)',
 'Requirement already satisfied: srsly<1.1.0,>=1.0.2 in /usr/local/lib/python3.6/dist-packages (from spacy>=2.2.2->en_core_web_lg==2.2.5) (1.0.5)',
 'Requirement already satisfied: plac<1.2.0,>=0.9.6 in /usr/local/lib/python3.6/dist-packages (from spacy>=2.2.2->en_core_web_lg==2.2.5) (1.1.3)',
 'Requirement already satisfied: blis<0.5.0,>=0.4.0 in /usr/local/lib/python3.6/dist-packages (from spacy>=2.2.2->en_core_web_lg==2.2.5) (0.4.1)',
 'Requirement already satisfied: cymem<2.1.

In [36]:

roberta.fill_mask('The first Star wars movie came out in <mask>', topk=3)
roberta.fill_mask('Vikram samvat calender is official in <mask>', topk=3)
roberta.fill_mask('<mask> is the common currency of the European Union', topk=3)

[(' that is the common currency of the European Union',
  0.15524248778820038,
  ' that'),
 (', is the common currency of the European Union', 0.10739845782518387, ','),
 (' the is the common currency of the European Union',
  0.08384165167808533,
  ' the')]

In [9]:
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.wsc', user_dir='examples/roberta/wsc')
#roberta.cuda()

roberta.disambiguate_pronoun('The _trophy_ would not fit in the brown suitcase because [it] was too big.')
# True
roberta.disambiguate_pronoun('The trophy would not fit in the brown _suitcase_ because [it] was too big.')
# False

roberta.disambiguate_pronoun('The city councilmen refused the demonstrators a permit because [they] feared violence.')
# 'The city councilmen'
roberta.disambiguate_pronoun('The city councilmen refused the demonstrators a permit because [they] advocated violence.')
# 'demonstrators'

doc = roberta.extract_features_aligned_to_words('I said, "hello RoBERTa."')
assert len(doc) == 10
for tok in doc:
  print('{:10}{} (...)'.format(str(tok), tok.vector[:5]))


/root/.cache/torch/hub/pytorch_fairseq_master/fairseq/models/roberta/hub_interface.py:176: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  masked_index = (tokens == self.task.mask_idx).nonzero()
Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


| dictionary: 50265 types
<s>       tensor([ 0.0545, -0.1485, -0.2556, -0.1266,  0.0687], grad_fn=<SliceBackward>) (...)
I         tensor([ 0.4806, -0.7086, -1.1670,  0.3662, -0.0280], grad_fn=<SliceBackward>) (...)
said      tensor([ 0.3193, -0.6057, -1.3547,  0.7873,  0.1528], grad_fn=<SliceBackward>) (...)
,         tensor([-0.1532, -0.7339, -1.0375, -0.0990,  0.2457], grad_fn=<SliceBackward>) (...)
"         tensor([ 0.2073,  0.2253, -0.9358, -0.1680,  0.2879], grad_fn=<SliceBackward>) (...)
hello     tensor([ 0.3492,  0.1469, -1.4918, -0.0900, -0.2172], grad_fn=<SliceBackward>) (...)
RoBERTa   tensor([-0.2418,  0.0366, -2.6119, -0.4323,  0.8791], grad_fn=<SliceBackward>) (...)
.         tensor([-0.3067, -0.1343, -0.7128, -0.3391,  0.3711], grad_fn=<SliceBackward>) (...)
"         tensor([-0.3067, -0.1343, -0.7128, -0.3391,  0.3711], grad_fn=<SliceBackward>) (...)
</s>      tensor([ 0.2398, -0.4107, -0.0265,  0.6106,  0.5240], grad_fn=<SliceBackward>) (...)


In [10]:
# !wget "https://gist.github.com/W4ngatang/60c2bdb54d156a41194446737ce03e2e/raw/1502038877f6a88c225a34450793fbc3ea87eaba/download_glue_data.py"
# !python download_glue_data.py --data_dir glue_data --tasks all

In [37]:
label_map = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}
ncorrect, nsamples = 0, 0

roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
roberta.eval()

with open('OneStopEnglishCorpus/Sentence-Aligned/ADV-ELE.txt') as fin:
  sent1 = fin.readline().strip()
  sent2 = fin.readline().strip()
  fin.readline() # spacer
  target = 'neutral'
  tokens = roberta.encode(sent1, sent2)
  prediction = roberta.predict('mnli', tokens).argmax().item()
  prediction_label = label_map[prediction]
  ncorrect += int(prediction_label == target)
  nsamples += 1

print('| Accuracy: ', float(ncorrect)/float(nsamples))

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


| Accuracy:  0.0
